In [ ]:
!pip install pyngrok #flask  tensorflow numpy pillow werkzeug

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from flask import Flask, request, jsonify, render_template
from pyngrok import ngrok
import tensorflow as tf
from PIL import Image
import numpy as np
import io

# Set your ngrok authtoken (Replace with your actual token)
ngrok.set_auth_token("2v8KGgFxY4RXsrz5CzIsAIDLrnY_22uM63C4uqHpAnDrJKDha")

# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/best_model.h5')

app = Flask(__name__, template_folder="/content/drive/My Drive/templates")

@app.route('/')
def home():
    return render_template("index.html")  # Serves the frontend

@app.route('/predict', methods=['POST'])
def predict():
    try:
        file = request.files['file']

        # Open image and convert to RGB (3 channels)
        image = Image.open(io.BytesIO(file.read())).convert('RGB')
        image = image.resize((128, 128))  # Resize to match model input

        # Convert image to numpy array and normalize
        image = np.array(image) / 255.0
        image = np.expand_dims(image, axis=0)  # Reshape to (1, 128, 128, 3)

        # Make prediction
        prediction = model.predict(image)
        result = "Cardiomegaly Detected" if prediction[0][0] > 0.5 else "No Cardiomegaly"

        return jsonify({'prediction': result, 'confidence': float(prediction[0][0])})

    except Exception as e:
        return jsonify({'error': str(e)})

# Open a ngrok tunnel to the Flask app on port 5000
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

# Run the Flask app
app.run(port=5000)


Public URL: NgrokTunnel: "https://4d58-35-203-183-108.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/May/2025 17:01:19] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/May/2025 17:01:20] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 591ms/step


INFO:werkzeug:127.0.0.1 - - [20/May/2025 17:01:37] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step


INFO:werkzeug:127.0.0.1 - - [20/May/2025 17:04:05] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step


INFO:werkzeug:127.0.0.1 - - [20/May/2025 17:04:16] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step


INFO:werkzeug:127.0.0.1 - - [20/May/2025 17:04:25] "POST /predict HTTP/1.1" 200 -
